In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

from modules.featureEng import *
from modules.modelEng import *

# to display entire data
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 150)

/Users/necronia/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
"""
Load Data
"""

train_df = pd.read_csv('data/train.csv', index_col='Id')
test_df = pd.read_csv('data/test.csv', index_col='Id')

In [3]:
"""
Feature Engineering
"""

train_X = train_df.loc[:,train_df.columns!='SalePrice']
train_Y = train_df['SalePrice']
test_X = test_df.copy()

### for testing ###
#from sklearn.model_selection import train_test_split
#train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, random_state=777)
###################

train_X, train_Y, test_X = pre_processing(train_X, train_Y, test_X)

In [4]:
"""
Predict house price
"""

elastic_net = grid_search(train_X, train_Y, ElasticNet())
lasso = grid_search(train_X, train_Y, Lasso())
ridge = grid_search(train_X, train_Y, Ridge())
random_forest = grid_search(train_X, train_Y, RandomForestRegressor())
support_vector_regressor = grid_search(train_X, train_Y, SVR())
XGBoost = grid_search(train_X, train_Y, xgb.XGBRegressor())
light_GBM = grid_search(train_X, train_Y, lgb.LGBMRegressor())

stacked_regression = StackingRegressor(
        regressors=[elastic_net, lasso, ridge, random_forest, XGBoost, light_GBM],
        meta_regressor=support_vector_regressor
)

stacked_regression.fit(train_X, train_Y)

stacked = stacked_regression.predict(test_X)

ensembled = np.expm1((0.1 * elastic_net.predict(test_X)) +
                     (0.1 * lasso.predict(test_X)) +
                     (0.1 * ridge.predict(test_X)) +
                     (0.05 * random_forest.predict(test_X)) +
                     (0.3 * XGBoost.predict(test_X)) +
                     (0.25 * light_GBM.predict(test_X)) +
                     (0.1 * stacked))

ensembled

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.0s finished


Model: ElasticNet Score: (0.13566143092301547) Best params: {'alpha': 0.0005, 'l1_ratio': 0.9, 'max_iter': 10000}
ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=10000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model: Lasso Score: (0.13570163888119588) Best params: {'alpha': 0.0005, 'normalize': False}
Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Model: Ridge Score: (0.13477452273562035) Best params: {'alpha': 10}
Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)
Fitting 5 folds for each of 27 candidates, 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  1.2min finished


Model: RandomForestRegressor Score: (0.16165170194209852) Best params: {'max_depth': 5, 'min_samples_leaf': 3, 'n_estimators': 1460}
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1460,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Model: XGBRegressor Score: (0.12433001656972636) Best params: {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 3000, 'refit': True, 'silent': True, 'subsample': 0.9}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=3000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             refit=True, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=True, subsample=0.9, verbosity=1)
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.9s finished


Model: LGBMRegressor Score: (0.13047931947721306) Best params: {'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 3000, 'num_leave': 1, 'objective': 'regression', 'refit': True}
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_bin=80,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=3000, n_jobs=-1, num_leave=1,
              num_leaves=31, objective='regression', random_state=None,
              refit=True, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


/Users/necronia/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


array([123871.2890255 , 155362.56590145, 183647.52591262, ...,
       157219.95063954, 115615.84903664, 224912.64147914])

In [5]:
### for testing ###
#RMSE = np.mean((ensembled - test_Y)**2)**(1/2) 
#print('Score : ' + str(RMSE))
###################

"""
Export submission data
"""
submission = pd.DataFrame({
    'Id':test_X.index,
    'SalePrice':ensembled
})
submission.to_csv('data/submission.csv', index=False)